### EVALUATION Statistichs with MLP

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTM_Emb_Classifier, EMG_Feature_Extractor
from utils.loaders import ActionNetEmgDataset
from torch.utils.data import DataLoader
from utils.logger import logger
from tqdm import tqdm
import pickle
import os
from utils.loaders import FeaturesExtendedEMGDataset
from models import MLP

/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: dlopen(/Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so, 0x0006): Symbol not found: __ZN3c1017RegisterOperatorsD1Ev
  Referenced from: <0EB69795-4559-3C98-9EA1-35B6A988BB99> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torchvision/image.so
  Expected in:     <E4E2FFCA-031E-3974-A7B0-45408D7F4956> /Users/andreavannozzi/GithubProjects/Multimodal-Egocentric-Action-Recognition/env/lib/python3.8/site-packages/torch/lib/libtorch_cpu.dylib
  warn(f"Failed to load image Python extension: {e}")


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from models import LSTM_Emb_Classifier, EMG_Feature_Extractor
from utils.loaders import ActionNetEmgDataset
from torch.utils.data import DataLoader
from utils.logger import logger
from tqdm import tqdm
import pickle
import os
from utils.loaders import FeaturesExtendedEMGDataset
from models import MLP

In [4]:
BATCH_SIZE = 32
LR = 0.008
MOMENTUM = 0.9
WEIGHT_DECAY = 1e-4
STEP_SIZE = 20
GAMMA = 0.1
NUM_EPOCHS = 50

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
if torch.backends.mps.is_available():
    DEVICE = 'mps'
    logger.info("------ USING APPLE SILICON GPU ------")

2024-06-08 18:30:15 LOG INFO ------ USING APPLE SILICON GPU ------


In [10]:
model = MLP(clip_feature_dim=16, num_class=20)
model = model.to(DEVICE)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=STEP_SIZE, gamma=GAMMA)

filepath = "saved_features/saved_feat_EMG_statistichs_25_dense_D1_train.pkl"
filepath2 = "saved_features/saved_feat_EMG_statistichs_25_dense_D1_test.pkl"

train_dataset = FeaturesExtendedEMGDataset(filepath)
loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=4, pin_memory=True, drop_last=True) # Inserisci il dataloader per il training

test_dataset = FeaturesExtendedEMGDataset(filepath2)
loader2 = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=4, pin_memory=True, drop_last=True)

logger.info(f"Model: {model}")
logger.info(f"len train_dataset: {len(train_dataset)}")
logger.info(f"len train_loader: {len(loader)}")

2024-06-08 18:32:29 LOG INFO Model: MLP(
  (classifier): Sequential(
    (0): ReLU()
    (1): Linear(in_features=16, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=20, bias=True)
  )
)
2024-06-08 18:32:29 LOG INFO len train_dataset: 8975
2024-06-08 18:32:29 LOG INFO len train_loader: 280


In [11]:
for x in loader:
    print(x[0].shape)
    print(x[1].shape)
    break

RuntimeError: _share_filename_: only available on CPU

In [7]:
model.train()
for epoch in range(NUM_EPOCHS):
        model.train()
        epoch_loss = [0.0, 0]
        for i_idx , x in tqdm(enumerate(train_loader1)):
            print(x.size())
            x = x.reshape(BATCH_SIZE, 5, 25, -1)
            x = x.permute(1, 0, 2, 3)
            y = y.to(DEVICE)
            #logger.info(f"X: {x[0][0]}")
            # Category Loss
            #logger.info(f"X: {x.size()}")
            for i in range(5):
                x_t = x[i].float().to(DEVICE)
                outputs, embeddings = model(x_t)
                # Log details about the outputs
                #logger.info(f"Output type: {cls_o.logits.shape}")
                loss = criterion(outputs, y.long())

                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                epoch_loss[0] += loss.item()
                epoch_loss[1] += x.size(0)

                if (i_val + 1) % (len(train_loader1) // 5) == 0:
                    logger.info("[{}/{}]".format(i + 1, len(train_loader1)))
            
        scheduler.step()
        logger.info(f'[EPOCH {epoch+1}] Avg. Loss: {epoch_loss[0] / epoch_loss[1]}')


        #save checkpoint in a file
        if (epoch+1) % 10 == 0:
            train_accuracy = evaluate(model, train_loader1, DEVICE)
            val_accuracy = evaluate(model, test_loader1, DEVICE)
            logger.info(f'[EPOCH {epoch+1}] Train Accuracy: {train_accuracy}')
            logger.info(f'[EPOCH {epoch+1}] Val Accuracy: {val_accuracy}')
            torch.save(model.state_dict(), f'./saved_models/LSTM_Emb_Classifier/final_LSTM_Emb_epoch_{epoch+1}.pth')
        if (epoch+1) % STEP_SIZE == 0:
            logger.info(f'Current LR: {scheduler.get_last_lr()}')





RuntimeError: _share_filename_: only available on CPU